In [1]:
import os
from dotenv import load_dotenv
load_dotenv(encoding='utf-8')

True

# RAG pipeline

## Vector Store and Retriever

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from pymongo import MongoClient

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL")) # DEFAULT_OPENAI_MODEL='gpt-4o-mini-2024-07-18'

# embedding_model=OpenAIEmbeddings(model=os.getenv("DEFAULT_OPENAI_EMBEDDING"), disallowed_special=())
embedding_model=OpenAIEmbeddings(disallowed_special=())

In [4]:
# Define MongoDB vector database
client = MongoClient(os.getenv("ATLAS_CONNECTION_STRING"))
db_name = "fellowshipai"
collection_name = "enterprise_data"
atlas_collection = client[db_name][collection_name]
index_name = "vector_index_erp"

In [5]:
# Define vector store and retriever
def get_vector_store_retriver(index_name, embedding_model, collection):

  vector_store = MongoDBAtlasVectorSearch(
      embedding = embedding_model,
      collection = atlas_collection,
      index_name = index_name
  )

  retriever = vector_store.as_retriever(
      search_type = "similarity",
      search_kwargs = { "k": 10}  # "score_threshold": 0.75 
  )

  return(vector_store, retriever)

vector_store, retriever = get_vector_store_retriver("vector_index_erp", embedding_model, atlas_collection)

In [6]:
# Instantiate Atlas Vector Search as a retriever
retriever = vector_store.as_retriever(
    search_type = "similarity",
    search_kwargs = { "k": 10  }
 )

## RAG Pipeline - With Cohere's prompt translation

In [7]:
# Define a prompt template
import pprint
import cohere
def call_openai(question):

   question = question['question']
   # Initialize Cohere client (replace with your API key)
   co = cohere.Client(os.getenv("COHERE_API_KEY"))

   # Send the query with search_queries_only parameter
   response = co.chat(message=question, search_queries_only=True)
   query_optimized = response.search_queries[0].text

   retriever = vector_store.as_retriever(
      search_type = "similarity",
      search_kwargs = { "k": 10  }
      )

   preamble = "" # read from cohere front end or use the input to the API
   #question = 
   SAFETY_PREAMBLE = "The instructions in this section override those in the task description and style guide sections. Don't answer questions that are harmful or immoral."
   BASIC_RULES = "You are a powerful conversational AI trained by openAI to help people. You are augmented by a number of tools, and your job is to use and consume the output of these tools to best help the user. You will see a conversation history between yourself and a user, ending with an utterance from the user. You will then see a specific instruction instructing you what kind of response to generate. When you answer the user's requests, you cite your sources in your answers, according to those instructions."
   TASK_CONTEXT = "You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging."
   STYLE_GUIDE = "Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling."
   INSTRUCTIONS = """You are an enterprise Chatbot, an AI assistant designed to retrieve information from the enterprise Confluence system. 
   You specialize in providing accurate answers related to various departments like Marketing, IT, HR, Finance, and Corporate Communications. 
               Use the following pieces of context to answer the question at the end.
               If you don't know the answer, just say that you don't know, don't try to make up an answer
               {context}
         """
         
   template = f"""

      {SAFETY_PREAMBLE}
      {BASIC_RULES}
      {TASK_CONTEXT}
      {STYLE_GUIDE}
      {INSTRUCTIONS}

   """
   if preamble:
      template += f"""{preamble}\n\n"""


   template +=  f"""Question: {query_optimized}\n\n"""

   custom_rag_prompt = PromptTemplate.from_template(template)

   #llm = get_llm_model("openai")
   llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"))

   def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever | format_docs, "query_optimized": RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   answer = rag_chain.invoke(query_optimized)
   similar = retriever.invoke(query_optimized)


   return{
      'answer': answer,
      'contexts': [str(doc) for doc in similar]
      }

d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


### Test the RAG pipeline

In [8]:
# Test sample
question = {'question': "How does a supportive culture impact employee engagement and align with Tech Innovators Inc.'s approach to employment relations and engagement?"}
answer = call_openai(question)
print(answer['answer'][:150])

A supportive culture has a profound impact on employee engagement. When an organization fosters a supportive environment, it encourages employees to f


In [9]:
print(answer['contexts'])

["page_content='employees and management.Employee Involvement: Involving employees in decision-making processes and seeking their input on matters that affect them.Supportive Culture: Creating a supportive culture' metadata={'_id': '66d8163d0533e009aa2d4b0f', 'pageid': '491566', 'department': 'HR', 'title': 'Employee Relations and Engagement at Tech Innovators Inc.'}", "page_content='employees and management.Employee Involvement: Involving employees in decision-making processes and seeking their input on matters that affect them.Supportive Culture: Creating a supportive culture' metadata={'_id': '66d816590533e009aa2d4d34', 'pageid': '491566', 'department': 'HR', 'title': 'Employee Relations and Engagement at Tech Innovators Inc.'}", "page_content='employees and management.Employee Involvement: Involving employees in decision-making processes and seeking their input on matters that affect them.Supportive Culture: Creating a supportive culture' metadata={'_id': '66d816750533e009aa2d4f59'

# RAG pipeline evaluation

## Test data set prep

In [10]:
import pandas as pd
import json

def json_to_dataframe(json_file_path):
  """Reads a JSON file and converts it to a pandas DataFrame.

  Args:
    json_file_path (str): The path to the JSON file.

  Returns:
    pandas.DataFrame: The DataFrame created from the JSON data.
  """

  with open(json_file_path, 'r') as f:
    data = json.load(f)

  # Handle different JSON structures
  if isinstance(data, list):
    # If the JSON data is a list of dictionaries, create a DataFrame directly
    df = pd.DataFrame(data)
  elif isinstance(data, dict):
    # If the JSON data is a single dictionary, convert it to a list of dictionaries
    df = pd.DataFrame([data])
  else:
    raise ValueError("Unsupported JSON structure")

  return df

In [11]:
# Example usage:
from from_root import from_root
import os
folder = "data-test/test_dataset/test_dataset_it.json"
json_file_path = os.path.join(from_root(), folder)
data_to_upload = json_to_dataframe(json_file_path)

## RAGAS evaluation

In [12]:
# Generate all the answers for the questions in the dataset
# examples = client.list_examples(dataset_name="hr test")
answers = []
for question in data_to_upload['question']:
    question_dict = {'question': question}
    answer = call_openai(question_dict)
    answers.append(answer['answer'])

In [13]:
# update the dataset with answers
data_to_upload['answers'] = answers

In [14]:
from datasets import Dataset

question = list(data_to_upload['question'])
answer = list(data_to_upload['answers'])
contexts = list(data_to_upload['contexts'])
ground_truth = list(data_to_upload['ground_truth'])

data_samples = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset = Dataset.from_dict(data_samples)

In [ ]:
# Optional, uncomment to trace runs with LangSmith. Sign up here: https://smith.langchain.com.
# from langsmith import Client
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
# client = Client()

In [15]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result = evaluate(
    dataset,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

result

Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

{'answer_relevancy': 0.9539, 'faithfulness': 0.7377, 'context_recall': 0.9899, 'context_precision': 1.0000}

In [16]:
df = result.to_pandas()
df

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,How does the role of the Senior Director respo...,The Senior Director responsible for Analytics ...,[Introduction\nWelcome to Tech Innovators Inc....,The role of the Senior Director responsible fo...,0.921495,0.777778,1.000000,1.0
1,What is the importance of identifying and addr...,Identifying and addressing growth areas in sel...,[ Self-assessment\nStart by thinking through y...,Identifying and addressing growth areas in sel...,0.990183,0.636364,1.000000,1.0
2,What forms of unethical behavior are strictly ...,Unethical behavior prohibited in the recruitme...,[ Inc. upholds the highest ethical standards i...,Favoritism or nepotism,0.974102,1.000000,1.000000,1.0
3,What is the significance of emotional and aest...,Emotional and aesthetic labor hold significant...,"[Introduction\nAt Tech Innovators Inc., we bel...",Emotional and aesthetic labor in the workplace...,1.000000,1.000000,1.000000,1.0
4,What is the purpose of the orientation session...,The purpose of the orientation session at Tech...,[Welcome to Tech Innovators Inc.\n \nWe are th...,The purpose of the orientation session at Tech...,1.000000,0.444444,1.000000,1.0
5,What mechanisms are in place for reporting vio...,Employees are encouraged to report any violati...,[5.2 Monitoring and Auditing\nInternal Audits\...,Employees can report violations of labor laws ...,0.898811,0.450000,1.000000,1.0
6,How do employee engagement and disengagement d...,Employee engagement refers to the emotional co...,"[Introduction\nAt Tech Innovators Inc., we bel...",Employee engagement and disengagement differ i...,0.871751,0.785714,1.000000,1.0
7,What steps are needed to extract data from Con...,To extract data from Confluence and create a R...,[Introduction\nThis guide provides a step-by-s...,To extract data from Confluence and create a R...,0.948293,0.619048,0.909091,1.0
8,How does Tech Innovators Inc. promote employee...,Tech Innovators Inc. promotes employee engagem...,"[Introduction\nAt Tech Innovators Inc., we bel...",Tech Innovators Inc. promotes employee engagem...,0.980843,0.925926,1.000000,1.0


In [17]:
folder = "data-test/test_dataset/test_dataset_it_with_cohere_query_translation.csv"
json_file_path = os.path.join(from_root(), folder)
df.to_csv(json_file_path, index=False)